Data is hard to visualize in Tableau given its organization.

Will try to do a few things:
1. Combine multiple lobbies into one long form
2. Combine multiple rounds into one round
3. Potentially combine multiple competitions into one

For now, we will start going step by step and then figuring out how to do the others.

# Preseason Qual Clean-up

We will clean up the preseason qualifiers

Right now we have the following file structure:

Region/Preseason_Qualifier_#/Round

What we want to do is iteratively combine each folder of files starting with the round files.
Then combine the rounds by each Preseason_Qualifier_# and then hopefully combine all those by region.

We will start with the APAC_North region and then move our way down alphabetically.

## Data Structure

First we have to understand how the data even looks when we load it into Python

In [1]:
# These are the necessary packages so far
import os
import re
import pandas as pd

In [13]:
# Start with the main directory
directory = '../Outputs/APAC_North/Preseason_Qualifier_1/Round_1'

df = pd.read_csv(f"{directory}/Lobby 1.csv")
df.reset_index()
df.head()

,Unnamed: 0,Standings,Standings.1,Standings.2,Standings.3
0,NaN,Team,Team.1,Total,Round 1
1,0.0,1.0,FC Destroy,112,112
2,1.0,2.0,Dps,38,38
3,2.0,3.0,CJAM,32,32
4,3.0,4.0,sspZ,31,31


Few things to note:
1. first column seems to be a repeat of standings. we can drop that.
2. total and round 1 are duplicate we can drop that
3. we can get rid of the current column header and replace it with the existing row 1

Lets start by cleaning that up

In [15]:
# drop unnecessary columns

df = df.drop(df.columns[[0,-1]],axis=1)

In [16]:
df.head()

,Standings,Standings.1,Standings.2
0,Team,Team.1,Total
1,1.0,FC Destroy,112
2,2.0,Dps,38
3,3.0,CJAM,32
4,4.0,sspZ,31


In [18]:
df.iloc[0,1:]

Standings.1    Team.1
Standings.2     Total
Name: 0, dtype: object

In [19]:
# Rename last 2 columns columns to be our first row

df = df.rename(columns = df.iloc[0, 1:]).loc[1:]

In [20]:
# Now it's looking better.

df.head()

,Standings,Team.1,Total
1,1.0,FC Destroy,112
2,2.0,Dps,38
3,3.0,CJAM,32
4,4.0,sspZ,31
5,5.0,Joel Mark,24


In [21]:
# Lets save to a new folder
if not os.path.exists(f'{directory}/Cleaned'): # check if the folder exists, otherwise make it
    os.mkdir(f'{directory}/Cleaned')

In [23]:
df.to_csv(f'{directory}/Cleaned/Round_1_cleaned.csv', index = False)

In [ ]:
# Time to repeat for all files in our folder
directory = '../Outputs/APAC_North/Preseason_Qualifier_1/Round_1'
    